In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

In [3]:
# Import and instantiate OpenAI embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [4]:
# Import and initialize Pinecone client
import pinecone
import os
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT') 
)
pinecone.whoami()


c:\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


WhoAmIResponse(username='7e6e9fd', user_label='langchain', projectname='32467cc')

In [5]:
# Find the existing index, clear for new start
index_name = "langchain-quickstart"
index=pinecone.Index(index_name)
index.delete(delete_all=True) # Clear the index first, then upload

{}

In [6]:
# Import parsers

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# Pinecone database: https://app.pinecone.io/organizations/-Nam3zmbSmzuXKeH8EWl/projects/us-west1-gcp-free:32467cc/indexes/langchain-quickstart
import glob

data_folder='../data/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

for doc in docs:
    print('Parsing: '+doc)
    loader = PyPDFLoader(doc)
    data = loader.load_and_split()
    
    # This is optional, but needed to play with the data parsing.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
    texts = text_splitter.split_documents(data)

    for text in texts:
        text.metadata['page']=text.metadata['page']+1   # Pages are 0 based, update
    
    print('Uploading to pinecone index '+index_name)
    vectorstore = Pinecone.from_documents(texts, embeddings_model, index_name=index_name)

Parsing: ../data\AMS_2014.pdf
Uploading to pinecone index langchain-quickstart
Parsing: ../data\AMS_2016.pdf
Uploading to pinecone index langchain-quickstart
Parsing: ../data\AMS_2018.pdf
Uploading to pinecone index langchain-quickstart
Parsing: ../data\AMS_2020.pdf
Uploading to pinecone index langchain-quickstart
